In [1]:
#r "D:\Development\VisualStudio\ITMO\ML\LinearRegression\bin\Release\netstandard2.1\LinearRegression.dll"
#r "nuget:XPlot.Plotly,3.0.1"    
#r "nuget:MathNet.Numerics,4.12.0"    

    
using System;
using System.IO;
using System.Linq;
using LinearRegression;
using System.Collections.Generic;
using XPlot.Plotly;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package MathNet.Numerics version 4.12.0

Installed package XPlot.Plotly version 3.0.1

In [2]:
private static DataSetObject ParseRow(String str)
{
    var row = str.Split(' ').Select(Double.Parse).ToArray();
    return new DataSetObject(row.Take(row.Length - 1).ToArray(), row[^1]);
}

var rows = File.ReadAllLines(@"D:\RandomTrash\linear\3.txt");

int objectCount = int.Parse(rows[1]);
var dataSet = rows
        .Skip(2)
        .Take(objectCount)
        .Select(ParseRow)
        .ToList();

var trainCount = Int32.Parse(rows[2 + objectCount]);
var trainDataSet = rows
        .Skip(2 + objectCount + 1)
        .Take(trainCount)
        .Select(ParseRow)
        .ToList();

trainDataSet = LinearRegression.LinearRegression.Normalize(trainDataSet);
dataSet = LinearRegression.LinearRegression.Normalize(dataSet);
double bestSmape = 101;
double regularization = 0.0001;
double bestRegularization = -1;
for(int i = 0; i < 100; i++)
{
    var mnkResult = LinearRegression.LinearRegression.GetWeightsWithMnk(dataSet, regularization);
    var trainSmape = LinearRegression.LinearRegression.GetSmape(trainDataSet, mnkResult);
    if(trainSmape < bestSmape)
    {
        bestSmape = trainSmape;
        bestRegularization = regularization;
    }
    
    regularization += 0.0005;
}
display("MNK :");
display("Best SMAPE : " + bestSmape + " with regularization lambda : " + bestRegularization);

MNK :

Best SMAPE : 0.09753675281720378 with regularization lambda : 0.0001

In [11]:
regularization = 0.001;
bestSmape = 101;

for(int i = 0; i < 100; i++)
{
    var gradientResult = LinearRegression.LinearRegression.GetWeightsWithGradient(dataSet, regularization, 2000);
    var trainSmape = LinearRegression.LinearRegression.GetSmape(trainDataSet, gradientResult);
    
    if(trainSmape < bestSmape)
    {
        bestSmape = trainSmape;
        bestRegularization = regularization;
    }
    
    regularization += 0.0005;
}

display("Gradient :");
display("Best SMAPE : " + bestSmape + " with regularization lambda : " + bestRegularization);

Gradient :

Best SMAPE : 0.07535698143980225 with regularization lambda : 0.03900000000000003

In [3]:
regularization = 0.0001;
bestSmape = 101;
List<double> trainSmapeResult = new List<double>(2000);
List<double> dataSetSmapeResult = new List<double>(2000);
List<int> stepCount = new List<int>(2000);
double prevSmape = 101;
for(int i = 1; i <= 2000; i++)
{
    var gradientResult = LinearRegression.LinearRegression.GetWeightsWithGradient(dataSet, regularization, i);
    var trainSmape = LinearRegression.LinearRegression.GetSmape(trainDataSet, gradientResult);
    var dataSetSmape = LinearRegression.LinearRegression.GetSmape(dataSet, gradientResult);
    
    dataSetSmapeResult.Add(dataSetSmape);
    trainSmapeResult.Add(trainSmape);
    stepCount.Add(i);
}


var scatters = new List<Graph.Scatter>
{
    new Graph.Scatter
    {
        name = "Train SMAPE",
        y = trainSmapeResult,
        x = stepCount
    },
    new Graph.Scatter
    {
        name = "DataSet SMAPE",
        y = dataSetSmapeResult,
        x = stepCount,
        marker = new Graph.Marker{size = 1}
    },
};
var chart = Chart.Plot(scatters);
chart.WithXTitle("Gradient count");
chart.WithYTitle("SMAPE value");
display(chart);